In [1]:
%load_ext autoreload
%autoreload 2
from models import load_model
model = load_model("llama_3_2_3b")

In [136]:
from agents.api import generate

main_prompt = "What should I do in Trondheim?"

output = generate(
    system_prompt="",
    prompt=main_prompt,
    model=model,
)
print(output)

Trondheim is a beautiful city located on the west coast of Norway, known for its rich history, stunning architecture, and vibrant cultural scene. Here are some top things to see and do when visiting Trondheim:

**Must-see attractions:**

1. **Nidaros Cathedral**: A magnificent Gothic cathedral built in 1130 over St Olav's grave.
2. **Old Town (Gamle Byen)**: Explore the charming streets, shops, and restaurants of this historic neighborhood.
3. **Trondheimsbrua Bridge**: Take a walk across Europe's longest pedestrian bridge for stunning views of the city.

**Outdoor activities:**

1. **Rafting or kayaking on Nidelva River**: Experience Norway's beautiful nature by paddling down one of Trondheim's most iconic rivers.
2. **Hiking in nearby forests and mountains**: Discover scenic trails, waterfalls, and picturesque villages just outside the city center.

**Museums:**

1. **Trondheims Museum (Nidarosmuseum)**: Learn about Trondheim's history from prehistoric times to modern days at this co

## Give access to tools...
We want to break down a query like this into three:
- Python interpreter: what date is it? (datetime lib)
- Weather API: what is the weather at *place* on *date*?
- Search agent: what are recommended activities at *place* on *date* in *weather*?


In [138]:
import datetime
import requests

def get_month():
    return datetime.datetime.now().strftime("%B")

print(get_month())


def get_weather(city):
    response = requests.get(f"http://wttr.in/{city}?format=%C")
    weather = ""
    if response.status_code == 200:
        weather = response.text.strip()
    else:
        return None

    class Weather(BaseModel):
        weather: str

    # here we do an LLM call to simplify the weather description
    human_weather = generate(
        system_prompt="",
        prompt=f"From the description of the weather conditions: '{weather}', rephrase it in very simple terms, e.g. rainy, cloudy, sunny, etc. Output in JSON.",
        schema=Weather.model_json_schema(),
        model=model,
        num_ctx=200,
        num_predict=100,
        temperature=0.0,
    )["weather"]
    return human_weather
    
print(get_weather("Trondheim"))

February
rainy


In [139]:
from util import WebSearch, get_html
from markitdown import MarkItDown

search = WebSearch()
md = MarkItDown()

def search(city: str, month: str, weather: str):
    query = f"What to do in {city} in {month} when it is {weather}?"
    print(query)
    urls = WebSearch().get_urls(query, max_results=1)
    soup = get_html(urls[0])
    with open("tmp.html", "w") as f:
        f.write(str(soup))
    result = md.convert("tmp.html")
    with open("tmp.md", "w") as f:
        f.write(result.text_content)
    return {
        "markdown": result.text_content,
        "query": query,
    }
    
result = search(place, get_month(), get_weather(place))

What to do in Trondheim in February when it is rainy?


In [163]:
from tqdm.notebook import tqdm  
def scan_results(result: str, query: str):
    hashed_results = result.split("\n#")
    hashed_results = [
        h.strip() for h in hashed_results if len(h.strip().split("\n")) > 1
    ]

    sys_prompt = f"You are an AI assistant tasked to best answer the user's query '{query}' based on the contents of the message. Perform an extractive retrieval and summarize the content before deciding whether it is related to the query (represented with relevant score on a scale of 0 to 10). The final answer should be a well-formed sentence capturing the essence of the text. If it is not relevant, the relevant score should be 0. Output in JSON"

    class QueryAnswer(BaseModel):
        relevant: int
        summary: str
        answer: str

    relevant_answers = []

    iterator = tqdm(hashed_results, desc="Processing results")
    for h in iterator:
        prompt = f"The content:\n{h}"
        output = generate(
            system_prompt=sys_prompt,
            prompt=prompt,
            schema=QueryAnswer.model_json_schema(),
            model=model,
            num_ctx=5000,
            num_predict=1000,
            temperature=0.0,
        )
        if not output:
            continue
        if output["relevant"] > 0:
            relevant_answers.append(output["answer"])
    
    return relevant_answers
    
final_results = scan_results(result["markdown"], result["query"])

Processing results:   0%|          | 0/29 [00:00<?, ?it/s]

In [164]:
final_results

['In February, visiting Trondheim offers numerous indoor attractions like museums (e.g., Ringve Museum), cathedrals (Nidaros Cathedral) that can be explored on rainy days.',
 'To make the most of your time in Trondheim in rainy February, consider using a trip planning app like Wanderlog to discover local experiences, stay informed about holiday events, dress accordingly for the wet weather, explore top attractions, and get insider tips from fellow travelers.',
 'Consider visiting the unique Ringve Museum in Trondheim during your trip for a fascinating experience exploring music and cultural heritage.',
 'When visiting Trondheim in February with rainy weather, consider exploring the stunning Nidaros Cathedral.',
 "Consider taking a scenic drive along the Golden Route to experience the natural beauty of Trondheim's countryside despite the rainy weather.",
 'Consider ice fishing on the Trondheim Fjord in February to enjoy tranquil surroundings and stunning views.',
 "For an engaging exper

In [176]:
from typing import List

def generate_summary(final_results: List[str], query: str):
    system_prompt = "You are an AI assistant designed to create step-wise summary of the information provided. Output in JSON."
    final_summary_prompt = f"For each element in the information, create a slightly shorter summary, based on the contents of the query '{query}'\nInformation:\n{'\n'.join(final_results)}"

    class Summary(BaseModel):
        fulltext: str
        summary: List[str]

    output = generate(
        system_prompt=system_prompt,
        prompt=final_summary_prompt,
        model=model,
        schema=Summary.model_json_schema(),
        num_ctx=5000,
        num_predict=1000,
        temperature=0.0,
    )
    if output:
        return output["summary"]
    return []

output = generate_summary(final_results, result["query"])
for s in output:
    print(f"- {s}")

- Visit museums
- Use a trip planning app
- Explore top attractions
- Dress for the weather
- Consider unique places like Ringve Museum
- Enjoy ice fishing on Trondheim Fjord
- Take guided tours of historic city center streets
- Visit indoor science museum
- Other indoor activities when no events


In [177]:
def full_search_pipeline(prompt: str):
    place = identify_place(prompt)
    month = get_month()
    weather = get_weather(place)
    result = search(place, month, weather)
    filtered_results = scan_results(result["markdown"], result["query"])
    output = generate_summary(filtered_results, result["query"])
    return output

print(full_search_pipeline("i'm in trondheim for a few days, what should I do?"))

What to do in Trondheim in February when it is rainy?


Processing results:   0%|          | 0/29 [00:00<?, ?it/s]

['Visit Ringve Museum', 'Explore Nidaros Cathedral', 'Drive Golden Route', 'Ice fish on Fjord', 'Guided city tour', 'Trondheim Science Museum visit']


In [203]:
from pydantic import Field
import time
from typing import Literal


sys_prompt = """
You are an AI assistant designed to think deeply and reason about the step-by-step process from the user's input, to create a logical separation for calling appropriate functions. Do not make assumptions, do not go beyond your given instructions and scope.

Your current database has the following information:
{current_memory}

You have access to the following functions IN SEQUENTIAL ORDER.
1. get_month() -> name of month
2. get_weather(city) -> weather forecast of today
3. search(city, month, weather) -> what to do in *city* in *month* when the weather is *weather*

Identify the current state of called functions, and toggle the appropriate flag to indicate completion.
Which function should be called next, based on the current status of the database? Respond with the function name and the parameters, along with an integer flag to indicate if the database is fully populated.
Output in JSON format.
"""


class NextFunction(BaseModel):
    function: Literal["get_month", "get_weather", "search"]
    parameters: dict = Field(default_factory=dict)


class FunctionSelector(BaseModel):
    next_fn: NextFunction
    is_complete: int = 0

current_memory = {"month": None, "weather": None, "search": None}
main_prompt = "What should I do in Trondheim?"

output = generate(
    system_prompt=sys_prompt.format(current_memory=current_memory),
    prompt=main_prompt,
    model=model,
    schema=FunctionSelector.model_json_schema(),
    num_ctx=300,
    num_predict=100,
    temperature=0.0, 
)
output

{'next_fn': {'function': 'get_month', 'parameters': {}}, 'is_complete': 0}

In [213]:
is_complete = False
current_memory = {"month": None, "weather": None, "search": None}
memory_map = {
    "get_month": "month",
    "get_weather": "weather",
    "search": "search",
}
fn_map = {
    "get_month": get_month,
    "get_weather": get_weather,
    "search": search,
}

def tool_search(prompt: str):
    res = None
    while True:
        tmp = generate(
            system_prompt=sys_prompt.format(current_memory=current_memory),
            prompt=prompt,
            model=model,
            schema=FunctionSelector.model_json_schema(),
            num_ctx=300,
            num_predict=100,
            temperature=0.0,
        )
        next_fn = tmp["next_fn"]
        print(f"Calling function: {next_fn['function']} with parameters: {next_fn['parameters']}")
        res = fn_map[next_fn["function"]](**next_fn["parameters"])
        current_memory[memory_map[next_fn["function"]]] = res
        if next_fn["function"] == "search":
            break
        print(f"Next memory: {current_memory}")
        print(res)
    
    
    filtered_results = scan_results(result["markdown"], result["query"])
    output = generate_summary(filtered_results, result["query"])
    return output

tool_search("What should I do in Trondheim?")

Calling function: get_month with parameters: {}
Next memory: {'month': 'February', 'weather': None, 'search': None}
February
Calling function: get_weather with parameters: {'city': 'Trondheim'}
Next memory: {'month': 'February', 'weather': 'rainy', 'search': None}
rainy
Calling function: search with parameters: {'city': 'Trondheim', 'month': 'February', 'weather': 'get_weather(Trondheim)'}
What to do in Trondheim in February when it is get_weather(Trondheim)?


Processing results:   0%|          | 0/29 [00:00<?, ?it/s]

['Visit museums',
 'Use a trip planning app',
 'Explore top attractions',
 'Dress for the weather',
 'Consider unique places like Ringve Museum',
 'Enjoy ice fishing on Trondheim Fjord',
 'Take guided tours of historic city center streets',
 'Visit indoor science museum',
 'Other indoor activities when no events']